***GENERATED CODE FOR 3mayclusccex PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CUST_ID', 'transformation_label': 'String Indexer'}], 'feature': 'CUST_ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '99', 'mean': '', 'stddev': '', 'min': 'C10001', 'max': 'C10103', 'missing': '0', 'distinct': '99'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CUST_ID'}, {'feature_label': 'CUST_ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CUST_ID')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        stageAttributes['TrainingPercent']


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run 3mayclusccexHooks.ipynb
try:
	#sourcePreExecutionHook()

	mayclusccexhdfs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/extra/tempFile1683120065.csv', 'filename': 'tempFile1683120065.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(mayclusccexhdfs)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run 3mayclusccexHooks.ipynb
try:
	#transformationPreExecutionHook()

	mayclusccexautofe = TransformationMain.run(mayclusccexhdfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "CUST_ID", "transformation_label": "String Indexer"}], "feature": "CUST_ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "99", "mean": "", "stddev": "", "min": "C10001", "max": "C10103", "missing": "0", "distinct": "99"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CUST_ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BALANCE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "2475.26", "stddev": "2438.67", "min": "3.78631", "max": "12136.21996", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BALANCE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BALANCE_FREQUENCY", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "0.91", "stddev": "0.18", "min": "0.090909", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BALANCE_FREQUENCY"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PURCHASES", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "992.2", "stddev": "1443.93", "min": "0.0", "max": "7091.01", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PURCHASES"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ONEOFF_PURCHASES", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "636.51", "stddev": "1188.11", "min": "0.0", "max": "6402.63", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ONEOFF_PURCHASES"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INSTALLMENTS_PURCHASES", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "355.69", "stddev": "629.19", "min": "0.0", "max": "3952.24", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INSTALLMENTS_PURCHASES"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CASH_ADVANCE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "1374.2", "stddev": "3292.58", "min": "0.0", "max": "27296.48576", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CASH_ADVANCE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PURCHASES_FREQUENCY", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "0.48", "stddev": "0.39", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PURCHASES_FREQUENCY"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ONEOFF_PURCHASES_FREQUENCY", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "0.2", "stddev": "0.3", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ONEOFF_PURCHASES_FREQUENC..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PURCHASES_INSTALLMENTS_FREQUENCY", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "0.34", "stddev": "0.39", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PURCHASES_INSTALLMENTS_FR..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CASH_ADVANCE_FREQUENCY", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "0.16", "stddev": "0.22", "min": "0.0", "max": "0.875", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CASH_ADVANCE_FREQUENCY"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CASH_ADVANCE_TRX", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "99", "mean": "3.47", "stddev": "6.4", "min": "0", "max": "37", "missing": "0"}, "updatedLabel": "CASH_ADVANCE_TRX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PURCHASES_TRX", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "99", "mean": "12.98", "stddev": "18.71", "min": "0", "max": "92", "missing": "0"}, "updatedLabel": "PURCHASES_TRX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CREDIT_LIMIT", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "99", "mean": "5347.98", "stddev": "3692.48", "min": "1000", "max": "16500", "missing": "0"}, "updatedLabel": "CREDIT_LIMIT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PAYMENTS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "2112.13", "stddev": "3793.05", "min": "0.0", "max": "28232.69446", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PAYMENTS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MINIMUM_PAYMENTS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "89", "mean": "1576.37", "stddev": "2892.63", "min": "73.203221", "max": "17862.55831", "missing": "10"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MINIMUM_PAYMENTS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PRC_FULL_PAYMENT", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "99", "mean": "0.09", "stddev": "0.23", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PRC_FULL_PAYMENT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TENURE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "99", "mean": "11.69", "stddev": "0.95", "min": "8", "max": "12", "missing": "0"}, "updatedLabel": "TENURE"}]}))

	#transformationPostExecutionHook(mayclusccexautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run 3mayclusccexHooks.ipynb
try:
	#mlPreExecutionHook()

	mayclusccexclustering = Clustering.run(mayclusccexautofe,spark,json.dumps( {"autoClustering": 0, "defaultclusters": 0, "TrainingPercent": "100", "model": "Kmeans", "originalfile": "/FileStore/platform/extra/tempFile1683120065.csv", "paascall": 1, "totalClusters": 2}))

	#mlPostExecutionHook(mayclusccexclustering)

except Exception as ex: 
	logging.error(ex)
